### Data cleaning

Este notebook sirve para limpiar la base de datos extraida y almcenada con título `scraped_data.csv`. Aquí se llevarán a cabo métodos de inspección del dataframe para comprobar que los datatypes sean correctos y que la información contenida sea útil y legible en otras herramientas como pueden ser SQL y PowerBI.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('../data/scraped_data.csv')
df.head()

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
0,2023-11-01 22:15:00,"32 47.4980 N, 9 54.3980 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,No,No,1,0,0,Spade,Sailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Night,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...
1,2023-10-31 07:50:00,"39 26.0000 N, 9 23.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,2,5,0,Twin rudder,Motoring,5 - 7,Rough,3 - 4 (7 - 16 knots),Day,50 - 75%,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by
2,2023-09-19 11:00:00,"37 40.0000 N, 8 54.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,No,Yes,1,0,0,Spade,Motoring,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...
3,2023-09-01 13:15:00,"45 36.0000 N, 3 45.0000 W","Yes, followed protocol, interaction lasted 10 ...",Sail,Over 15m,Not towing,Yes,Yes,1,2,0,Spade,Sailing,3 - 4,Calm,3 - 4 (7 - 16 knots),Day,25 - 50%,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...
4,2023-09-02 03:45:00,"42 45.0000 N, 9 14.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,1,2,0,Spade,Motorsailing,5 - 7,Calm,0 - 2 (0 - 6 knots),Night,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   date                           154 non-null    object
 1   lat_and_long                   154 non-null    object
 2   GTOA_Protocol                  154 non-null    object
 3   Boat_Type                      154 non-null    object
 4   Boat_Length                    154 non-null    object
 5   Towing_Inflatable              154 non-null    object
 6   Trailing_Fishing_Lure          154 non-null    object
 7   Physical_Contact_With_Boat     154 non-null    object
 8   Number_of_Adult_Orcas          154 non-null    object
 9   Number_of_Juvenile_Orcas       154 non-null    object
 10  Number_of_Uncertain_Age_Orcas  154 non-null    object
 11  Rudder                         154 non-null    object
 12  Motoring_or_Sailing            154 non-null    object
 13  Speed

Tenemos 28 columnas, todas de tipo objeto por como se ha escrapeado la información en el notebook `1.WebScraping`. Entre otras cosas se va a hacer lo siguiente:

* Borrar aquellas filas que no contengan información o que estén repetidas
* Corregir cualquier fila de datos que pueda estar en columnas equivocadas
* Separar datos en dos columnas si fuera necesario, como por ejemplo con latitud y longitud que actualmente se encuentra en una sola columna
* Cambiar los datatypes de las columnas
* Quitar duplicados en caso de que los hubiera
* Ya que la mayor parte de las respuestas del formulario que llevaron a cabo los patrones de las embarcaciones que sufrieron una interacción con una orca constaban de varias opciones (y no respuesta libre), será util convertir las variables categóricas en numéricas, vía aplicación de One-Hot encoding. Esto supondrá un paso clave de cara a aplicar métodos de modelos predictivos en el futuro.
* Identificar y rechazar outliers en caso de que los hubiera y siempre razonando si interesa deshacernos de ellos o nos aporta una información útil.
* Comprobación de consistencia de la BBDD.

#### 1. Descripción de las columnas

A continuación se incluye una descripción de las columnas con las que cuenta originalmente nuestra base de datos:

* **date**: Fecha y hora aproximada de la interacción
* **lat_and_long**: Latitud y longitud donde se dio la interacción
* **GTOA_Protocol**: Se siguió el *protocolo* de GT Orca Atlántica: Arriar las velas, detener la embarcación, apagar el motor y mantener un perfil bajo así como *duración* de la interacción
* **Boat_Type**: Tipo de barco - Velero | Motor | Barco de pesca
* **Boat_Length**: Eslora del barco (m) - menos de 10m | 10-12.5 | 12.5-15m | Más de 15m
* **Towing_Inflatable**: Se encontraba el barco remolcando una lancha neumática?
* **Trailing_Fishing_Lure**: Se encontraba el barco arrastrando un señuelo de pesca?
* **Physical_Contact_With_Boat**: Hubo contacto físico de las orcas con la embarcación?
* **Number_of_Adult_Orcas**: Número de orcas adultas?
* **Number_of_Juvenile_Orcas**: Número de orcas juveniles?
* **Number_of_Uncertain_Age_Orcas**: Número de orcas de edad incierta?
* **Rudder**: Tipo de timón - Spade/pala | Semi skeg/Semicompensado | Full skeg/Completo | Twin rudder/Doble timón | Keel hung/Quilla corrida
* **Motoring_or_Sailing**: Motor o vela - Vela | Motor | Motor/Vela | Hove-to
* **Speed_Knots**: Velocidad (kts)
* **Sea_State**: Estado del mar - Calma | Moderado | Gruesa
* **Wind_Speed_Beaufort**: Velocidad del viento (Escala de Beaufort) - 0.2 | 3-4 | 5-6 | 7+
* **Daylight_or_Darkness**: Noche/Día - Amanecer | Día | Atardecer | Noche
* **Cloud_Cover**: Cobertura de nubes - 0-25% | 25-50% | 50-75% | 75-100%
* **Distance_Off_Land_NM**: Distancia a tierra (nm) - 0-2 | 2-5 | 5-10 | Más de 10
* **Depth_Meters**: Profundidad (m) - hasta 20m | 20-40m | 40-200m | 200m+
* **Depth_Gauge**: Medidor de profundidad - On | Off
* **Autopilot**: Piloto automático - On | Off
* **Hull_Topsides_Color**: Color del casco - Blanco | Oscuro
* **Antifoul_Color**: Color del antifoul - Negro | Azul | Rojo | Blanco | Verde | Coppercoat | Otro
* **Boat_Damaged**: Fue dañado el barco o necesita reparación alguna? Sí, menor | Sí, moderado | Sí, grandes reparaciones | No
* **Tow_Required**: ¿Fue remolcado? - Yes | No
* **Crew_Response**: Descripción abierta de la interacción así como acciones que se tomaron y su estas  disuadieron o no la interacción con las orcas.
* **Orcas_Behaviour**: Descripción del comportamiento de la/s orca/s

In [9]:
df.GTOA_Protocol.value_counts()

GTOA_Protocol
No, did not follow protocol, interaction lasted less than 10 minutes    40
Yes, followed protocol, interaction lasted 10 to 30 minutes             26
Yes, followed protocol, interaction lasted less than 10 minutes         24
No, did not follow protocol, interaction lasted 10 to 30 minutes        24
Yes, followed protocol, interaction lasted 30 to 60 minutes             15
Yes, followed protocol, interaction lasted more than 60 minutes         11
No, did not follow protocol, interaction lasted 30 to 60 minutes         9
No, did not follow protocol, interaction lasted more than 60 minutes     3
Sail                                                                     1
Unknown                                                                  1
Name: count, dtype: int64

#### Fila 105
* Cambio en la fila con indice 105 por error de dimensionamiento - Vamos a mover una celda a la derecha todos los valores desde la columna 'Rudder' en adelante:

In [15]:
index_to_shift = 105

# Get the index of the 'Rudder' column
rudder_column_index = df.columns.get_loc('Rudder')

# Use the shift method for the specified row and 'Rudder' column onwards
df.iloc[index_to_shift, rudder_column_index:] = df.iloc[index_to_shift, rudder_column_index:].shift(1)

print(df.iloc[index_to_shift])

date                                                           2022-04-13 15:15:00
lat_and_long                                              35 52.0000 N, 6 1.1000 W
GTOA_Protocol                                                                 Sail
Boat_Type                                                               12.5 - 15m
Boat_Length                                                             Not towing
Towing_Inflatable                                                            Spade
Trailing_Fishing_Lure                                                      Unknown
Physical_Contact_With_Boat                                                 Unknown
Number_of_Adult_Orcas                                                            0
Number_of_Juvenile_Orcas                                                         0
Number_of_Uncertain_Age_Orcas                                                    0
Rudder                                                                        None
Moto

Ahora voy a cambiar a mano:
* GTOA_Protocol: 'Unknown'
* Boat_Type: 'Sail'
* Boat_Length: '12.5 - 15m'
* Towing_Inflatable: 'Not towing'
* Rudder: 'Spade'

In [17]:
# Voy a usar .loc[] para hacer los cambios
df.loc[105, 'GTOA_Protocol'] = 'Unknown'
df.loc[105, 'Boat_Type'] = 'Sail'
df.loc[105, 'Boat_Length'] = '12.5 - 15m'
df.loc[105, 'Towing_Inflatable'] = 'Not towing'
df.loc[105, 'Rudder'] = 'Spade'

print(df.iloc[105])

date                                                           2022-04-13 15:15:00
lat_and_long                                              35 52.0000 N, 6 1.1000 W
GTOA_Protocol                                                              Unknown
Boat_Type                                                                     Sail
Boat_Length                                                             12.5 - 15m
Towing_Inflatable                                                       Not towing
Trailing_Fishing_Lure                                                      Unknown
Physical_Contact_With_Boat                                                 Unknown
Number_of_Adult_Orcas                                                            0
Number_of_Juvenile_Orcas                                                         0
Number_of_Uncertain_Age_Orcas                                                    0
Rudder                                                                       Spade
Moto

#### Fila 153
* Vamos a borrar esta fila por contener todo 'Unknown'

In [20]:
df.drop(index=153, inplace=True)
df.tail()

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour,Boat_type
148,2022-07-06 07:15:00,"35 59.0000 N, 5 55.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,Unknown,Unknown,0,0,0,Twin rudder,Motorsailing,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,No,No,We were following another sail boat from Porti...,"They appeared from my starboard, went under th...",NaN
149,2022-07-03 18:30:00,"39 24.6240 N, 9 38.4150 W","Yes, followed protocol, interaction lasted 10 ...",Sail,10 - 12.5m,Not towing,Unknown,Unknown,0,0,0,Twin rudder,Sailing,5 - 7,Moderate,3 - 4 (7 - 16 knots),Day,50 - 75%,Over 10,200m+,On,On,White/light,Black,"Yes, extensive - major works required",No,Nous naviguions à la voile avec un catamaran d...,"Relativement calme, allant sous le bateau, don...",NaN
150,2022-07-03 17:50:00,"38 7.2100 N, 9 1.4300 W","Yes, followed protocol, interaction lasted les...",Sail,Under 10m,Not towing,Unknown,Unknown,0,0,0,Spade,Motorsailing,5 - 7,Calm,5 - 6 (17 - 27 knots),Day,25 - 50%,5 - 10,40 - 200m,On,On,White/light,Blue,No,No,I only saw 2 adult females and 2 juveniles. My...,Very placid and almost lethargic. I’ve never s...,NaN
151,2022-07-03 13:00:00,"39 21.0000 N, 9 35.0000 W","No, did not follow protocol, interaction laste...",Sail,Over 15m,Not towing,Unknown,Unknown,0,0,0,Twin rudder,Motorsailing,5 - 7,Moderate,7+ (28 knots+),Day,25 - 50%,5 - 10,40 - 200m,On,Off,White/light,Black,"Yes, extensive - major works required",No,The boat is a catamaran with 2 engines and rud...,There were 3 adults and 2 juveniles. The small...,NaN
152,2022-06-24 12:50:00,"36 50.0000 N, 8 55.0000 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,Unknown,Unknown,0,0,0,Spade,Sailing,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,The two orcas were nearby a buoy and as soon a...,The two orcas (one of 6 meters and other of 3-...,NaN


## Comprobación columna a columna
* Vamos a comprobar los valores unicos de cada columna para detectar valores que no cuadren

#### GTOA_Protocol

In [21]:
df.GTOA_Protocol.unique()

array(['No, did not follow protocol, interaction lasted less than 10 minutes',
       'Yes, followed protocol, interaction lasted less than 10 minutes',
       'Yes, followed protocol, interaction lasted 10 to 30 minutes',
       'Yes, followed protocol, interaction lasted 30 to 60 minutes',
       'No, did not follow protocol, interaction lasted 10 to 30 minutes',
       'Yes, followed protocol, interaction lasted more than 60 minutes',
       'No, did not follow protocol, interaction lasted more than 60 minutes',
       'No, did not follow protocol, interaction lasted 30 to 60 minutes',
       'Unknown'], dtype=object)

Dado que esta columna contiene estos valores únicos, podemos facilmente definir una función que nos separe en dos columnas de mayor utilidad. Por un lado nos están diciendo si se siguió o no el protocolo y por otro cuanto tiempo duró la interacción. Separaremos esta columna en las siguientes: 'Followed_GTOA_Protocol' y 'Interaction_time'.

Vamos a aprovechar como están construidas las respuestas para separar en cada una de las nuevas columnas:
* 'Followed_GTOA_Protocol': A partir de la primera palabra podemos saber si se siguió (``Yes``) si no (``No``) o si no se conoce la respuesta del patrón, desconocido (``Unknown``)

* 'Interaction_time': A partir de las últimas 4 palabras podemos clasificar en 5 rangos de tiempo:
1) less than 10m
2) 10-30m
3) 30-60m
4) more than 60m
5) Unknown

In [22]:
# Definimos una función que nos categorice:
def saca_protocolo_tiempo(value):
    
    # Empezamos categorizando según se siguiera el protocolo o no
    if value.startswith('Yes'):
        protocolo = 'Yes'
    elif value.startswith('No'):
        protocolo = 'No'
    else:
        protocolo = 'Unknown'

    # Seguimos categorizando por duración de la interacción
    if 'less than 10 minutes' in value:
        interaccion = '0-10'
    elif '10 to 30 minutes' in value:
        interaccion = '10-30'
    elif '30 to 60 minutes' in value:
        interaccion = '30-60'
    elif 'more than 60 minutes' in value:
        interaccion = '60+'
    else:
        interaccion = 'Unknown'

    return pd.Series([protocolo, interaccion], index = ['Followed_GTOA_Protocol', 'Interaction_time'])

In [24]:
# Hago la llamada a la función con un apply() y creo las dos nuevas columnas:
df[['Followed_GTOA_Protocol', 'Interaction_time']] = df.GTOA_Protocol.apply(saca_protocolo_tiempo)
df.head()

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour,Boat_type,Followed_GTOA_Protocol,Interaction_time
0,2023-11-01 22:15:00,"32 47.4980 N, 9 54.3980 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,No,No,1,0,0,Spade,Sailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Night,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...,NaN,No,0-10
1,2023-10-31 07:50:00,"39 26.0000 N, 9 23.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,2,5,0,Twin rudder,Motoring,5 - 7,Rough,3 - 4 (7 - 16 knots),Day,50 - 75%,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by,NaN,Yes,0-10
2,2023-09-19 11:00:00,"37 40.0000 N, 8 54.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,No,Yes,1,0,0,Spade,Motoring,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...,NaN,No,0-10
3,2023-09-01 13:15:00,"45 36.0000 N, 3 45.0000 W","Yes, followed protocol, interaction lasted 10 ...",Sail,Over 15m,Not towing,Yes,Yes,1,2,0,Spade,Sailing,3 - 4,Calm,3 - 4 (7 - 16 knots),Day,25 - 50%,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...,NaN,Yes,10-30
4,2023-09-02 03:45:00,"42 45.0000 N, 9 14.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,1,2,0,Spade,Motorsailing,5 - 7,Calm,0 - 2 (0 - 6 knots),Night,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...,NaN,Yes,0-10


## MAÑANA SEGUIR CON:
* Comprobar si ejecutando de nuevo el codigo se va la antepenultima columna esa de nans que fue un typo mio
* Comprobar si las dos nuevas columnas de protocolo y tiempo están bien hechas
* Si esto es todo para GTOA, borrar la columna ppal, reordenar (poner las nuevas columnas al principio) y pasar a la siguiente columna
* Vamos!!!

In [12]:
df_copia = df.copy()
df_copia.head()

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
0,2023-11-01 22:15:00,"32 47.4980 N, 9 54.3980 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,No,No,1,0,0,Spade,Sailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Night,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...
1,2023-10-31 07:50:00,"39 26.0000 N, 9 23.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,2,5,0,Twin rudder,Motoring,5 - 7,Rough,3 - 4 (7 - 16 knots),Day,50 - 75%,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by
2,2023-09-19 11:00:00,"37 40.0000 N, 8 54.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,No,Yes,1,0,0,Spade,Motoring,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...
3,2023-09-01 13:15:00,"45 36.0000 N, 3 45.0000 W","Yes, followed protocol, interaction lasted 10 ...",Sail,Over 15m,Not towing,Yes,Yes,1,2,0,Spade,Sailing,3 - 4,Calm,3 - 4 (7 - 16 knots),Day,25 - 50%,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...
4,2023-09-02 03:45:00,"42 45.0000 N, 9 14.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,1,2,0,Spade,Motorsailing,5 - 7,Calm,0 - 2 (0 - 6 knots),Night,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...


In [13]:
index_to_shift = 105

# Get the index of the 'Rudder' column
rudder_column_index = df_copia.columns.get_loc('Rudder')

# Use the shift method for the specified row and 'Rudder' column onwards
df_copia.iloc[index_to_shift, rudder_column_index:] = df_copia.iloc[index_to_shift, rudder_column_index:].shift(1)

In [14]:
df_copia

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
0,2023-11-01 22:15:00,"32 47.4980 N, 9 54.3980 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,No,No,1,0,0,Spade,Sailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Night,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...
1,2023-10-31 07:50:00,"39 26.0000 N, 9 23.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,2,5,0,Twin rudder,Motoring,5 - 7,Rough,3 - 4 (7 - 16 knots),Day,50 - 75%,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by
2,2023-09-19 11:00:00,"37 40.0000 N, 8 54.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,No,Yes,1,0,0,Spade,Motoring,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...
3,2023-09-01 13:15:00,"45 36.0000 N, 3 45.0000 W","Yes, followed protocol, interaction lasted 10 ...",Sail,Over 15m,Not towing,Yes,Yes,1,2,0,Spade,Sailing,3 - 4,Calm,3 - 4 (7 - 16 knots),Day,25 - 50%,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...
4,2023-09-02 03:45:00,"42 45.0000 N, 9 14.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,1,2,0,Spade,Motorsailing,5 - 7,Calm,0 - 2 (0 - 6 knots),Night,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...
5,2023-08-22 11:50:00,"42 53.7600 N, 9 21.3900 W","No, did not follow protocol, interaction laste...",Sail,Over 15m,Not towing,Yes,Yes,5,0,0,Spade,Sailing,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Black,No,No,"We stayed calm, made no unnecessary noise and ...",The animals seemed to be curious and intereste...
6,2023-08-17 13:30:00,"35 52.7000 N, 5 38.6000 W","No, did not follow protocol, interaction laste...",Sail,Over 15m,Not towing,Yes,Yes,6,0,0,Twin rudder,Motorsailing,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,200m+,On,On,White/light,Blue,No,No,Pinger; no effect Fireworks; effective,"Came from stern side, submerged under stern an..."
7,2023-07-15 23:58:00,"36 15.0090 N, 5 11.2660 W","Yes, followed protocol, interaction lasted 30 ...",Sail,12.5 - 15m,Not towing,No,Yes,4,0,0,Sailing,5 - 7,Moderate,3 - 4 (7 - 16 knots),Night,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Blue,"Yes, moderate - immediate repairs required",Yes,We were sailing and the boat was on autopilot....,One of them was under the boat. Others were at...,"Not applicable, did not reverse"
8,2023-08-09 11:30:00,"38 12.0000 N, 9 4.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,Yes,Yes,1,0,0,Full skeg,Motoring,5 - 7,Calm,0 - 2 (0 - 6 knots),Day,0 - 25%,Over 10,40 - 200m,On,On,Dark colour,Black,"Yes, minor - will wait until the end of the se...",No,"As we encountered the orca, we stopped the eng...",Swimming around a lot. O distance about 100 me...
9,2023-07-21 16:10:00,"36 24.6000 N, 4 51.2000 W","No, did not follow protocol, interaction laste...",Sail,Over 15m,Not t